In [2]:
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from multiprocessing import Pool

In [3]:
def read_large_file(file_path):
    """파일에서 텍스트를 읽어 리스트로 반환."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()


In [4]:
def calculate_similarity(chunk):
    """주어진 텍스트 조각에 대해 유사도를 계산."""
    indices, texts = chunk
    vectorizer = TfidfVectorizer().fit_transform(texts)
    similarity_matrix = cosine_similarity(vectorizer)
    results = []
    for i in range(len(texts)):
        for j in range(i + 1, len(texts)):
            if similarity_matrix[i, j] > 0.8:  # 유사도가 0.8 이상인 경우
                results.append((indices[i], indices[j], similarity_matrix[i, j]))
    return results

In [5]:
def process_in_chunks(file_path, chunk_size):
    """파일을 청크 단위로 분할."""
    texts = read_large_file(file_path)
    num_texts = len(texts)
    chunks = [
        (list(range(i, min(i + chunk_size, num_texts))),
         texts[i:min(i + chunk_size, num_texts)])
        for i in range(0, num_texts, chunk_size)
    ]
    return chunks

In [6]:
def test_chunk_sizes(file_path, chunk_sizes):
    """청크 크기별 실행 시간을 측정."""
    results = []
    for chunk_size in chunk_sizes:
        print(f"Testing chunk size: {chunk_size}")
        start_time = time.time()

        # 파일을 청크 단위로 나눔
        chunks = process_in_chunks(file_path, chunk_size)

        # 멀티프로세싱을 사용해 청크별로 유사도 계산
        with Pool() as pool:
            pool.map(calculate_similarity, chunks)

        elapsed_time = time.time() - start_time
        print(f"Chunk size {chunk_size} completed in {elapsed_time:.2f} seconds")
        results.append((chunk_size, elapsed_time))
    return results

In [7]:
def save_test_results(results, output_file):
    """청크 크기 테스트 결과를 파일에 저장."""
    with open(output_file, 'w', encoding='utf-8') as file:
        for chunk_size, elapsed_time in results:
            file.write(f"Chunk Size: {chunk_size}, Time: {elapsed_time:.2f} seconds\n")

In [ ]:
if __name__ == "__main__":
    input_file = '복무Q&A.txt'  # 입력 텍스트 파일 경로
    output_file = 'chunk_size_test_results.txt'  # 테스트 결과 저장 파일
    chunk_sizes = [100, 500, 1000, 2000, 5000]  # 테스트할 청크 크기 목록

    # 청크 크기별 테스트 실행
    test_results = test_chunk_sizes(input_file, chunk_sizes)

    # 결과 저장
    save_test_results(test_results, output_file)

    print(f"Test results saved in {output_file}")

Testing chunk size: 100
